In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from BSDE import *
import warnings 
import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show, output_file
import time
warnings.filterwarnings("ignore")

In [3]:
T = 0.25
K = 100
S0 = 100
sigma = 0.2
r = 0.01
mu = 0.05
R = 0.06
q = 0.

In [4]:
M_run = 20
m = 12
N = 100000
deg = 8
n_picard = 10

In [5]:
a = np.zeros(M_run)
timing = np.zeros_like(a)
for i in range (M_run): 
    start = time.time()
    test = BSDE(S0, K, T, mu, sigma, q)
    a[i] = test.get_price_lsm(R, r, N, m, n_picard=n_picard, deg=deg, oPayoff='call combination')
    elapsed = time.time() - start
    timing[i] = round(elapsed, 3)

In [6]:
import pandas as pd

In [7]:
param = ['q', 'mu', 'R ', 'r ','sigma', 'T', 'K', 'S0']
values = [q, mu, R, r, sigma, T, K, S0]
dataset = list(zip(param,values))
df_param = pd.DataFrame(data = dataset, columns=['Parameters', 'values'])

In [8]:
sparam = ['N_particles', 'm_discretization', 'N_run', 'n_picard','polynomial degree']
svalues = [N, m, M_run, n_picard, deg]
sp_dataset = list(zip(sparam,svalues))
df_sparam = pd.DataFrame(sp_dataset, columns=['Specific Parameters', 'values'])
df_sparam

,Specific Parameters,values
0,N_particles,100000
1,m_discretization,12
2,N_run,20
3,n_picard,10
4,polynomial degree,8


In [9]:
run_id = ["run {}".format(i) for i in range(1, M_run+1)]
df = pd.DataFrame(list(zip(run_id, a, timing)), columns = ['run','values', 'time'])

In [10]:
min_a = round(df['values'].min(), 4)
max_a = round(df['values'].max(), 4)
mean_a = round(df['values'].mean(), 4)
std_a = round(df['values'].std(), 4)
left = round(mean_a - 1.96 * std_a / M_run, 4)
right = round(mean_a + 1.96 * std_a / M_run, 4)

In [11]:
stat_param = ['mean', 'std', '95% confidence interval', 'min', 'max']
stat_values = [mean_a, std_a, [left, right], min_a, max_a]
stat_data = list(zip(stat_param, stat_values))
stat_df = pd.DataFrame(stat_data, columns = ['stat parameter', 'values'])
stat_df

,stat parameter,values
0,mean,2.9381
1,std,0.0154
2,95% confidence interval,"[2.9366, 2.9396]"
3,min,2.9159
4,max,2.9787


In [12]:
df_join = pd.concat([df_param, df_sparam, df, stat_df], axis = 0)
df_join.to_csv('output_lsm.csv')